# **STEMfit example**

This notebook gives a short demonstration of how to detect atoms in a STEM image and determine their positions using center of mass mode. First, we start by importing `STEMfit`:

In [ ]:
using Pkg; Pkg.activate("./path/to/environment");
import STEMfit

Now, we load our example image. The image we use here is of three oxide perovskite layers grown epitaxially on each other. By setting `convert=true`, we let `STEMfit` convert our 16 bit TIFF image into the required format.

In [ ]:
image = STEMfit.load_image("image.tif")

We filter the raw image using a combination of singular value decomposition and Gaussian convolution. `STEMfit` will automatically determine the number of singular vectors to use. For diagnostic purposes, we can set the keyword argument `plot=true`:

In [ ]:
filtered_image = STEMfit.filter_image(image, plot=false)

Then, we use the `find_atoms` function to find atoms in the image using an adaptive thresholding procedure. The function takes the image as an argument as well as `bias` and `window_size` parameters which control the thresholding. Increasing `bias` tends to make the threshold higher (i.e. more separation of atoms), `window_size` determines how big of an area is considered at each point in the image. The function returns a matrix of atom positions, a vector of atom widths, a vector of atom intensities and the thresholded image. The latter can be used to assess the quality of the thresholding and asjust the `bias` and `window_size` parameters. 

In [ ]:
tp = STEMfit.ThresholdingParameters(bias=0.5, window_size=5, minimum_atom_size=0)

In [ ]:
(atom_parameters, thresholded_image) = STEMfit.find_atoms(filtered_image, tp);

# **Processing**

Having now found the atomic positions in the image, we can find a unit cell. The `uc_allowed_angles` parameter controls the permissible range of unit cell angles, i.e. `uc_allowed_angles ≈ 90` for orthogonal unit cells. `uc_allowed_orientation` controls the range of orientations the unit cell can have in the image, where zero is the direction horizontally to the left. These parameters can be used to make sure the 'correct' unit cell is found or they can be omitted. 

In [ ]:
(unit_cells, neighbors, atom_tree) = STEMfit.find_unit_cells(
                                    atom_parameters, 
                                    uc_allowed_angles=85:95, #Only orthogonal unit cells
                                    uc_allowed_orientation=175:185 #Only horizontal unit cells
                                    );

The unit cell is now used to find the local lattice parameters, in units of pixels, for each atomic column.

In [ ]:
results = STEMfit.calculate_lattice_parameters(atom_parameters, unit_cells[1])

To convert pixel units into true length units (e.g. nanometers), we need to find the pixel size. We can use part of the image of which we know the true lattice parameters as a reference. In this case, the bottom 20% of the image is a single crystal of SrTiO3, which has a cubic structure with lattice parameters of 0.3905 nm. 

In [ ]:
known_lattice_parameters = (0.3905, 0.3905)

reference_range_y = 0.8*size(image, 1):size(image, 1) #Bottom 20% of the image
reference_range_x = 0:size(image, 2) #Full width of the image

STEMfit.add_pixel_size(
    results, 
    reference_range_y, 
    reference_range_x, 
    known_lattice_parameters
    )

Finally, we save the results to a file. The pixel units are automatically converted into length units if a pixel size is available in `results`. We also save the thresholded image and the filtered image.

In [ ]:
STEMfit.save_results("Results/results.csv", results)

STEMfit.save("Results/binarized image.png", thresholded_image)
STEMfit.save("Results/filtered image.png", filtered_image)